In [ ]:
import sempy.fabric as fabric
import sempy_labs as sempy_labs
from sempy_labs import migration, report, directlake
from sempy_labs import lakehouse as lake
from sempy_labs.tom import connect_semantic_model
import pandas as pd
import pyspark.sql.functions
from delta.tables import DeltaTable
from pyspark.sql.functions import lit, current_timestamp
from datetime import datetime
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, TimestampType

StatementMeta(, de6cac60-1d58-4427-b852-06cf5d487c95, 6, Finished, Available, Finished)

In [ ]:
prm_starttime = datetime.utcnow()

StatementMeta(, f5a63c90-6ea8-424d-af33-281ccea157e1, 8, Finished, Available, Finished)

2025-09-10 09:01:50.824310


In [ ]:
def fnc_startUp(_Tablename):

    # Define Lakehouse name and description.
    LH_Name = "SemanticLink_Lakehouse"
    LH_Name = "Lkh_SemanticLink"
    LH_desc = "Lakehouse for Power BI usage monitoring"

    # Setup log table voor refrhes opvolging van de notebooks
    fnc_NotebookRefreshTableCheck() 

    # Truncate or drop the table to have a clear start to insert into  
    fnc_TableCheckStartOfRun(_Tablename, 'Truncate')

    # Mount the Lakehouse for direct file system access
    lakehouse = mssparkutils.lakehouse.get(LH_Name)
    mssparkutils.fs.mount(lakehouse.get("properties").get("abfsPath"), f"/{LH_Name}")

    # Retrieve and store local and ABFS paths of the mounted Lakehouse
    local_path = mssparkutils.fs.getMountPath(f"/{LH_Name}")
    lh_abfs_path = lakehouse.get("properties").get("abfsPath")
    return lh_abfs_path

In [ ]:
def fnc_TableCheckStartOfRun(_Tablename, _TruncateOrDrop):
    if spark.catalog.tableExists(_Tablename):
        if(_TruncateOrDrop == 'Truncate'):
            spark.sql(f'TRUNCATE table {_Tablename}')
        if(_TruncateOrDrop == 'Delete'):
            spark.sql(f'DROP table {_Tablename}')


In [ ]:
def fnc_NotebookRefreshTableCheck():
    if spark.catalog.tableExists('refreshTimes_Notebooks'):
        print('refreshTimes_Notebooks Existed')
    else:    
        schema = StructType([
            StructField("NotebookName", StringType(), False),
            StructField("StartTime", TimestampType(), True),
            StructField("EndTime", TimestampType(), True)
        ])
        df = spark.createDataFrame([], schema)
        df.write.format("delta").saveAsTable('refreshTimes_Notebooks')  

In [ ]:
def fnc_PrepareColumns(_A):
    _A.columns = _A.columns.str.replace('[^a-zA-Z0-9]', '', regex=True)
    _A.columns = _A.columns.str.replace('[ ]', '', regex=True)
    return _A

In [14]:
def fnc_LognotebookRefresh(_NotebookName, _StartTime, _EndTime):
    spark.sql(f"INSERT INTO refreshTimes_Notebooks (NotebookName, StartTime, EndTime) VALUES ('{_NotebookName}', '{_StartTime}', '{_EndTime}')")

StatementMeta(, de6cac60-1d58-4427-b852-06cf5d487c95, 18, Finished, Available, Finished)